In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings 
warnings.filterwarnings(action='ignore')


import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM, InputLayer
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics 
from tensorflow import keras
import random  
from tensorflow.keras.callbacks import EarlyStopping

seed_num = 42
# os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
gpus = tf.config.experimental.list_physical_devices('GPU')
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape


def get_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                 loss = "binary_crossentropy", metrics=['acc'])
    return lstm

from keras.wrappers.scikit_learn import KerasClassifier

class MyKerasClassifier(KerasClassifier):
    def fit(self, x, y, sample_weight=None, **kwargs):
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        #---------------수정---------------
        if sample_weight is not None:
            print('sample weight : ', sample_weight)
            if sample_weight[0] == 0.00016175994823681658:
                print('x, y', x.shape, x.sum().sum())
                return super(MyKerasClassifier, self).fit(x, y, **kwargs)
            weights = sample_weight / sum(sample_weight)
            random_range = [(sum(weights[:i]), sum(weights[:i])+weights[i]) if i!=0 else (0, weights[i]) for i in range(len(weights))]
            random_nums = [random.random() for _ in range(len(weights))]
            idx_list = []
            for i in random_nums:
                for j in random_range:
                    if j[0] < i <= j[1]:
                        idx_list.append(random_range.index(j))
                        break
            new_x = x[idx_list, :, :]
            new_y = y[idx_list]
            print(new_x.sum().sum())
            print('new_x, new_y', new_x.shape, new_y.shape)
            return super(MyKerasClassifier, self).fit(new_x, new_y, **kwargs)
        
    def predict(self, x, **kwargs):
        return super(MyKerasClassifier, self).predict(x)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


Using TensorFlow backend.


In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings 
warnings.filterwarnings(action='ignore')


import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, LSTM, InputLayer
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn import metrics 
from tensorflow import keras
import random  
from tensorflow.keras.callbacks import EarlyStopping

seed_num = 42
# os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
gpus = tf.config.experimental.list_physical_devices('GPU')
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape


def get_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
    lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(Dropout(0.2))
    lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
    lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(units=1, activation='sigmoid'))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                 loss = "binary_crossentropy", metrics=['acc'])
    return lstm

from keras.wrappers.scikit_learn import KerasClassifier
import random
class MyKerasClassifier(KerasClassifier):
    def fit(self, x, y, sample_weight=None, validation_split=0.25, **kwargs):
        y = np.array(y)
        if len(y.shape) == 2 and y.shape[1] > 1:
            self.classes_ = np.arange(y.shape[1])
        elif (len(y.shape) == 2 and y.shape[1] == 1) or len(y.shape) == 1:
            self.classes_ = np.unique(y)
            y = np.searchsorted(self.classes_, y)
        else:
            raise ValueError('Invalid shape for y: ' + str(y.shape))
        self.n_classes_ = len(self.classes_)
        #---------------수정---------------
        if sample_weight is not None:
            print('sample weight : ', sample_weight)
            if sample_weight[0] == 0.00016175994823681658:
                print('x, y', x.shape, x.sum().sum())
                return super(MyKerasClassifier, self).fit(x, y, **kwargs)
            print('x sum', x.sum().sum())
            
            idx = np.arange(len(x))
            random.shuffle(idx)
            i = int(len(x)*0.75)
            train_x, train_y = x[idx[:i],:,:], y[idx[:i]]
            val_x, val_y = x[idx[i:],:,:], y[idx[i:]]
            
            
            train_sw, val_sw = sample_weight[idx[:i]], sample_weight[idx[i:]]
            train_sw, val_sw = train_sw/sum(train_sw)*len(train_sw), val_sw/sum(val_sw)*len(val_sw)
            
            weights = val_sw / sum(val_sw)
            random_range = [(sum(weights[:i]), sum(weights[:i])+weights[i]) if i!=0 else (0, weights[i]) for i in range(len(weights))]
            random_nums = [random.random() for _ in range(len(weights))]
            idx_list = []
            for i in random_nums:
                for j in random_range:
                    if j[0] < i <= j[1]:
                        idx_list.append(random_range.index(j))
                        break
            new_val_x = val_x[idx_list, :, :]
            new_val_y = val_y[idx_list]
            
            kwargs['validation_data'] = (new_val_x, new_val_y)
            kwargs['sample_weight'] = train_sw
            return super(MyKerasClassifier, self).fit(train_x, train_y, **kwargs)
        
        return super(MyKerasClassifier, self).fit(x, y, **kwargs)
    
    
    
    def predict(self, x, **kwargs):
        return super(MyKerasClassifier, self).predict(x)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# Adaboost-LSTM

## Single LSTM

In [36]:
X_train.shape

(6182, 10, 4068)

In [32]:
idx = np.arange(7727)
random.shuffle(idx)
idx

array([7267, 6914,  990, ..., 2575, 3450, 1696])

In [6]:
with tf.device('/device:GPU:0'):
    print("Single LSTM Start")
    model = get_model()
    
    early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
    cb_checkpoint = ModelCheckpoint(filepath='./models/single_lstm.h5', monitor='val_loss',
                                    verbose=1, save_best_only=True)
    model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.25, callbacks=[early_stop, cb_checkpoint])
    preds = model.predict(X_test)

    preds[preds>0.5]=1
    preds[preds<=0.5]=0
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)
    acc = accuracy_score(y_test, preds)

    print(f'accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

Single LSTM Start
Epoch 1/50
37/37 [==============================] - 6s 81ms/step - loss: 0.6736 - acc: 0.6055 - val_loss: 0.6657 - val_acc: 0.6177
Epoch 2/50
37/37 [==============================] - 3s 70ms/step - loss: 0.6696 - acc: 0.6100 - val_loss: 0.6509 - val_acc: 0.6177
Epoch 3/50
37/37 [==============================] - 3s 70ms/step - loss: 0.6045 - acc: 0.6706 - val_loss: 0.5288 - val_acc: 0.7510
Epoch 4/50
37/37 [==============================] - 3s 70ms/step - loss: 0.4900 - acc: 0.7821 - val_loss: 0.4942 - val_acc: 0.7639
Epoch 5/50
37/37 [==============================] - 3s 71ms/step - loss: 0.4187 - acc: 0.8261 - val_loss: 0.4894 - val_acc: 0.7743
Epoch 6/50
37/37 [==============================] - 3s 70ms/step - loss: 0.3760 - acc: 0.8503 - val_loss: 0.5036 - val_acc: 0.7743
Epoch 7/50
37/37 [==============================] - 3s 71ms/step - loss: 0.3591 - acc: 0.8613 - val_loss: 0.5329 - val_acc: 0.7587
Epoch 8/50
37/37 [==============================] - 3s 71ms/step 

In [11]:
%%time
from tensorflow.keras.callbacks import ModelCheckpoint

with tf.device('/device:GPU:0'):
    early_stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1, restore_best_weights=True)
#     cb_checkpoint = ModelCheckpoint(filepath='./models/adaboost_lstm.h5', monitor='val_acc',
#                                     verbose=1, save_best_only=True)
    base_estimator = MyKerasClassifier(build_fn=get_model, epochs=300, batch_size=10000,
                                       validation_split=0.25, callbacks=[early_stop])
#     base_estimator = MyKerasClassifier(build_fn=get_model, epochs=50, batch_size=128, validation_split=0.25)

    boosted_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=60, random_state=42, learning_rate=1.2)
    
    print("Adaboost LSTM Start")
    boosted_classifier.fit(X_train, y_train)
    preds = boosted_classifier.predict(X_test)

    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)
    acc = accuracy_score(y_test, preds)

    print(f'Adaboost accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

Adaboost LSTM Start
sample weight :  [0.00016176 0.00016176 0.00016176 ... 0.00016176 0.00016176 0.00016176]
x, y (6182, 10, 4068) 1644947.0
Epoch 1/70
2/2 [==============================] - 5s 538ms/step - loss: 0.6814 - acc: 0.6104 - val_loss: 0.6666 - val_acc: 0.6177
Epoch 2/70
2/2 [==============================] - 0s 134ms/step - loss: 0.6782 - acc: 0.6091 - val_loss: 0.6652 - val_acc: 0.6177
Epoch 3/70
2/2 [==============================] - 0s 141ms/step - loss: 0.6753 - acc: 0.6061 - val_loss: 0.6658 - val_acc: 0.6177
Epoch 4/70
2/2 [==============================] - 0s 135ms/step - loss: 0.6721 - acc: 0.6025 - val_loss: 0.6665 - val_acc: 0.6177
Epoch 5/70
2/2 [==============================] - 0s 135ms/step - loss: 0.6741 - acc: 0.6033 - val_loss: 0.6661 - val_acc: 0.6177
Epoch 6/70
2/2 [==============================] - 0s 133ms/step - loss: 0.6744 - acc: 0.6040 - val_loss: 0.6654 - val_acc: 0.6177
Epoch 7/70
2/2 [==============================] - 0s 136ms/step - loss: 0.6745 

2/2 [==============================] - 0s 135ms/step - loss: 0.3527 - acc: 0.8654 - val_loss: 0.5245 - val_acc: 0.7671
Epoch 58/70
2/2 [==============================] - 0s 126ms/step - loss: 0.3355 - acc: 0.8773 - val_loss: 0.5448 - val_acc: 0.7691
Epoch 59/70
2/2 [==============================] - 0s 137ms/step - loss: 0.3444 - acc: 0.8708 - val_loss: 0.5369 - val_acc: 0.7574
Epoch 60/70
2/2 [==============================] - 0s 136ms/step - loss: 0.3426 - acc: 0.8717 - val_loss: 0.5372 - val_acc: 0.7652
Epoch 61/70
2/2 [==============================] - 0s 125ms/step - loss: 0.3239 - acc: 0.8857 - val_loss: 0.5394 - val_acc: 0.7678
Epoch 62/70
2/2 [==============================] - 0s 134ms/step - loss: 0.3149 - acc: 0.8889 - val_loss: 0.5398 - val_acc: 0.7639
Epoch 63/70
2/2 [==============================] - 0s 136ms/step - loss: 0.3164 - acc: 0.8874 - val_loss: 0.5596 - val_acc: 0.7613
Epoch 64/70
2/2 [==============================] - 0s 128ms/step - loss: 0.3104 - acc: 0.8900 -

Epoch 44/70
1/1 [==============================] - 0s 172ms/step - loss: 0.6863 - acc: 0.3998 - val_loss: 0.6775 - val_acc: 0.3814
Epoch 45/70
1/1 [==============================] - 0s 177ms/step - loss: 0.6852 - acc: 0.3955 - val_loss: 0.6775 - val_acc: 0.3814
Epoch 46/70
1/1 [==============================] - 0s 189ms/step - loss: 0.6826 - acc: 0.3957 - val_loss: 0.6775 - val_acc: 0.3814
Epoch 47/70
1/1 [==============================] - 0s 177ms/step - loss: 0.6843 - acc: 0.3995 - val_loss: 0.6776 - val_acc: 0.3814
Epoch 48/70
1/1 [==============================] - 0s 188ms/step - loss: 0.6865 - acc: 0.3995 - val_loss: 0.6778 - val_acc: 0.3814
Epoch 49/70
1/1 [==============================] - 0s 180ms/step - loss: 0.6828 - acc: 0.3966 - val_loss: 0.6781 - val_acc: 0.3814
Epoch 50/70
1/1 [==============================] - 0s 186ms/step - loss: 0.6858 - acc: 0.3963 - val_loss: 0.6784 - val_acc: 0.3814
Epoch 51/70
1/1 [==============================] - 0s 183ms/step - loss: 0.6842 - a

1/1 [==============================] - 0s 179ms/step - loss: 0.6964 - acc: 0.5982 - val_loss: 0.6949 - val_acc: 0.5841
Epoch 31/70
1/1 [==============================] - 0s 156ms/step - loss: 0.6991 - acc: 0.5959 - val_loss: 0.6942 - val_acc: 0.5841
Epoch 32/70
1/1 [==============================] - 0s 151ms/step - loss: 0.6970 - acc: 0.5959 - val_loss: 0.6934 - val_acc: 0.5841
Epoch 33/70
1/1 [==============================] - 0s 156ms/step - loss: 0.6929 - acc: 0.5951 - val_loss: 0.6928 - val_acc: 0.5841
Epoch 34/70
1/1 [==============================] - 0s 169ms/step - loss: 0.6960 - acc: 0.5864 - val_loss: 0.6921 - val_acc: 0.5841
Epoch 35/70
1/1 [==============================] - 0s 184ms/step - loss: 0.7001 - acc: 0.5804 - val_loss: 0.6916 - val_acc: 0.5841
Epoch 36/70
1/1 [==============================] - 0s 179ms/step - loss: 0.6958 - acc: 0.5818 - val_loss: 0.6912 - val_acc: 0.5841
Epoch 37/70
1/1 [==============================] - 0s 143ms/step - loss: 0.6926 - acc: 0.5801 -

Epoch 17/70
1/1 [==============================] - 0s 183ms/step - loss: 0.7002 - acc: 0.5580 - val_loss: 0.6927 - val_acc: 0.6100
Epoch 18/70
1/1 [==============================] - 0s 180ms/step - loss: 0.7046 - acc: 0.5499 - val_loss: 0.6900 - val_acc: 0.6100
Epoch 19/70
1/1 [==============================] - 0s 183ms/step - loss: 0.6997 - acc: 0.5372 - val_loss: 0.6876 - val_acc: 0.6100
Epoch 20/70
1/1 [==============================] - 0s 180ms/step - loss: 0.7056 - acc: 0.5240 - val_loss: 0.6856 - val_acc: 0.6100
Epoch 21/70
1/1 [==============================] - 0s 176ms/step - loss: 0.6952 - acc: 0.5298 - val_loss: 0.6839 - val_acc: 0.6100
Epoch 22/70
1/1 [==============================] - 0s 171ms/step - loss: 0.7072 - acc: 0.4984 - val_loss: 0.6826 - val_acc: 0.3900
Epoch 23/70
1/1 [==============================] - 0s 176ms/step - loss: 0.7016 - acc: 0.4820 - val_loss: 0.6814 - val_acc: 0.3900
Epoch 24/70
1/1 [==============================] - 0s 178ms/step - loss: 0.6951 - a

...

Epoch 67/70
1/1 [==============================] - 0s 179ms/step - loss: 0.6802 - acc: 0.4907 - val_loss: 0.7423 - val_acc: 0.3874
Epoch 68/70
1/1 [==============================] - 0s 182ms/step - loss: 0.6809 - acc: 0.4855 - val_loss: 0.7423 - val_acc: 0.3874
Epoch 69/70
1/1 [==============================] - 0s 183ms/step - loss: 0.6756 - acc: 0.4938 - val_loss: 0.7423 - val_acc: 0.3874
Epoch 70/70
1/1 [==============================] - 0s 179ms/step - loss: 0.6792 - acc: 0.4783 - val_loss: 0.7423 - val_acc: 0.3874
sample weight :  [1.78583226e-04 6.62426227e-05 5.83843908e-05 ... 5.77810736e-05
 5.90635099e-05 5.45246360e-05]
x sum 1644947.0
Epoch 1/70
1/1 [==============================] - 5s 5s/step - loss: 0.7172 - acc: 0.3995 - val_loss: 0.7152 - val_acc: 0.3822
Epoch 2/70
1/1 [==============================] - 0s 157ms/step - loss: 0.7058 - acc: 0.4070 - val_loss: 0.7090 - val_acc: 0.3822
Epoch 3/70
1/1 [==============================] - 0s 184ms/step - loss: 0.6972 - acc: 0.4

1/1 [==============================] - 0s 176ms/step - loss: 0.6941 - acc: 0.4881 - val_loss: 0.7032 - val_acc: 0.3822
Epoch 54/70
1/1 [==============================] - 0s 185ms/step - loss: 0.6935 - acc: 0.4829 - val_loss: 0.7031 - val_acc: 0.3822
Epoch 55/70
1/1 [==============================] - 0s 183ms/step - loss: 0.6973 - acc: 0.4889 - val_loss: 0.7031 - val_acc: 0.3822
Epoch 56/70
1/1 [==============================] - 0s 176ms/step - loss: 0.6932 - acc: 0.4771 - val_loss: 0.7031 - val_acc: 0.3822
Epoch 57/70
1/1 [==============================] - 0s 181ms/step - loss: 0.6933 - acc: 0.4921 - val_loss: 0.7030 - val_acc: 0.3822
Epoch 58/70
1/1 [==============================] - 0s 154ms/step - loss: 0.6898 - acc: 0.4838 - val_loss: 0.7030 - val_acc: 0.3822
Epoch 59/70
1/1 [==============================] - 0s 162ms/step - loss: 0.6887 - acc: 0.4996 - val_loss: 0.7030 - val_acc: 0.3822
Epoch 60/70
1/1 [==============================] - 0s 180ms/step - loss: 0.6938 - acc: 0.4909 -

Epoch 40/70
1/1 [==============================] - 0s 179ms/step - loss: 0.6759 - acc: 0.4967 - val_loss: 0.7599 - val_acc: 0.3840
Epoch 41/70
1/1 [==============================] - 0s 188ms/step - loss: 0.6724 - acc: 0.4987 - val_loss: 0.7602 - val_acc: 0.3840
Epoch 42/70
1/1 [==============================] - 0s 173ms/step - loss: 0.6723 - acc: 0.5007 - val_loss: 0.7606 - val_acc: 0.3840
Epoch 43/70
1/1 [==============================] - 0s 195ms/step - loss: 0.6735 - acc: 0.4958 - val_loss: 0.7609 - val_acc: 0.3840
Epoch 44/70
1/1 [==============================] - 0s 192ms/step - loss: 0.6720 - acc: 0.5007 - val_loss: 0.7611 - val_acc: 0.3840
Epoch 45/70
1/1 [==============================] - 0s 153ms/step - loss: 0.6739 - acc: 0.4771 - val_loss: 0.7614 - val_acc: 0.3840
Epoch 46/70
1/1 [==============================] - 0s 181ms/step - loss: 0.6747 - acc: 0.4930 - val_loss: 0.7614 - val_acc: 0.3840
Epoch 47/70
1/1 [==============================] - 0s 176ms/step - loss: 0.6729 - a

1/1 [==============================] - 0s 183ms/step - loss: 0.7118 - acc: 0.4820 - val_loss: 0.6568 - val_acc: 0.3917
Epoch 27/70
1/1 [==============================] - 0s 179ms/step - loss: 0.7120 - acc: 0.4846 - val_loss: 0.6570 - val_acc: 0.3917
Epoch 28/70
1/1 [==============================] - 0s 178ms/step - loss: 0.7127 - acc: 0.4803 - val_loss: 0.6570 - val_acc: 0.3917
Epoch 29/70
1/1 [==============================] - 0s 170ms/step - loss: 0.7115 - acc: 0.4840 - val_loss: 0.6570 - val_acc: 0.3917
Epoch 30/70
1/1 [==============================] - 0s 191ms/step - loss: 0.7110 - acc: 0.4628 - val_loss: 0.6567 - val_acc: 0.3917
Epoch 31/70
1/1 [==============================] - 0s 185ms/step - loss: 0.7152 - acc: 0.4697 - val_loss: 0.6564 - val_acc: 0.3917
Epoch 32/70
1/1 [==============================] - 0s 184ms/step - loss: 0.7082 - acc: 0.4970 - val_loss: 0.6561 - val_acc: 0.3917
Epoch 33/70
1/1 [==============================] - 0s 166ms/step - loss: 0.7155 - acc: 0.4789 -

Epoch 13/70
1/1 [==============================] - 0s 146ms/step - loss: 0.7038 - acc: 0.4245 - val_loss: 0.7000 - val_acc: 0.3917
Epoch 14/70
1/1 [==============================] - 0s 176ms/step - loss: 0.7014 - acc: 0.4357 - val_loss: 0.6982 - val_acc: 0.3917
Epoch 15/70
1/1 [==============================] - 0s 183ms/step - loss: 0.6966 - acc: 0.4513 - val_loss: 0.6964 - val_acc: 0.3917
Epoch 16/70
1/1 [==============================] - 0s 159ms/step - loss: 0.6957 - acc: 0.4533 - val_loss: 0.6949 - val_acc: 0.3917
Epoch 17/70
1/1 [==============================] - 0s 179ms/step - loss: 0.7007 - acc: 0.4478 - val_loss: 0.6935 - val_acc: 0.3917
Epoch 18/70
1/1 [==============================] - 0s 145ms/step - loss: 0.7013 - acc: 0.4665 - val_loss: 0.6924 - val_acc: 0.3917
Epoch 19/70
1/1 [==============================] - 0s 152ms/step - loss: 0.6929 - acc: 0.4705 - val_loss: 0.6915 - val_acc: 0.3917
Epoch 20/70
1/1 [==============================] - 0s 177ms/step - loss: 0.6942 - a

In [4]:
a = [i for i in boosted_classifier.staged_predict_proba(X_test)]
len(a[0])
a

[array([[0.2707135 , 0.7292865 ],
        [0.07444376, 0.92555624],
        [0.04450702, 0.955493  ],
        ...,
        [0.0898302 , 0.9101698 ],
        [0.11763357, 0.8823664 ],
        [0.23773056, 0.76226944]], dtype=float32),
 array([[0.40409738, 0.5959026 ],
        [0.24302235, 0.7569776 ],
        [0.19565073, 0.80434924],
        ...,
        [0.26112202, 0.73887795],
        [0.29030725, 0.7096927 ],
        [0.3814843 , 0.6185157 ]], dtype=float32),
 array([[0.4275194 , 0.5724806 ],
        [0.31221482, 0.6877852 ],
        [0.27394864, 0.72605133],
        ...,
        [0.32597783, 0.6740222 ],
        [0.34771815, 0.6522818 ],
        [0.4120743 , 0.5879257 ]], dtype=float32),
 array([[0.44819024, 0.5518098 ],
        [0.35904843, 0.64095163],
        [0.32766244, 0.6723376 ],
        ...,
        [0.36968783, 0.6303122 ],
        [0.386842  , 0.613158  ],
        [0.4362423 , 0.56375766]], dtype=float32),
 array([[0.45480263, 0.54519737],
        [0.3826213 , 0.6173788

In [5]:
aa = [i for i in boosted_classifier.staged_predict(X_test)]
aa

[array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1, 1, 1, ..., 1, 1, 1]),
 array([1,

In [6]:
acc_list = [accuracy_score(y_test,i) for i in boosted_classifier.staged_predict(X_test)]
acc_list

[0.7637540453074434,
 0.7650485436893204,
 0.7624595469255664,
 0.7637540453074434,
 0.7637540453074434,
 0.7631067961165049,
 0.7631067961165049,
 0.7644012944983819,
 0.7637540453074434,
 0.7624595469255664,
 0.7572815533980582,
 0.7618122977346279,
 0.7618122977346279,
 0.7611650485436893,
 0.7585760517799353,
 0.7598705501618123,
 0.7637540453074434,
 0.7579288025889968,
 0.7618122977346279,
 0.7585760517799353,
 0.7585760517799353,
 0.7611650485436893,
 0.7592233009708738,
 0.7579288025889968,
 0.7592233009708738,
 0.7618122977346279,
 0.7592233009708738,
 0.7598705501618123,
 0.7579288025889968,
 0.7624595469255664,
 0.7579288025889968,
 0.7618122977346279,
 0.7579288025889968,
 0.7618122977346279,
 0.7585760517799353,
 0.7585760517799353,
 0.7592233009708738,
 0.7579288025889968,
 0.7631067961165049,
 0.7592233009708738,
 0.7579288025889968,
 0.7572815533980582,
 0.7572815533980582,
 0.7598705501618123,
 0.7598705501618123,
 0.7598705501618123,
 0.7611650485436893,
 0.7618122977

In [26]:
y_test

array([1, 0, 1, ..., 1, 1, 1])

In [ ]:
boosted_classifier.estimator_errors_

- .fit(new_x, new_y) : Adaboost accuracy : 0.7501618122977346, precision : 0.7742914979757085, recall : 0.8243534482758621, f1 : 0.7985386221294364, roc_auc : 0.7314635960990331
- .fit(new_x, new_y, **kwargs) : accuracy : 0.7430420711974111, precision : 0.7734294541709578, recall : 0.8092672413793104, f1 : 0.7909426013691417, roc_auc : 0.7263516109651819
- .fit(new_x, new_y, sample_weight=sample_weight) : 


In [5]:
y_pred_test = boosted_classifier.predict(X_test)

y_pred_test[y_pred_test>0.5]=1
y_pred_test[y_pred_test<=0.5]=0
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
roc_auc = roc_auc_score(y_test, y_pred_test)
acc = accuracy_score(y_test, y_pred_test)

print(f'accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

accuracy : 0.7430420711974111, precision : 0.7734294541709578, recall : 0.8092672413793104, f1 : 0.7909426013691417, roc_auc : 0.7263516109651819


## 모델저장

In [8]:
import pickle
import joblib

joblib.dump(boosted_classifier, './models/adaboost_lstm.pkl') 
reload_model = joblib.load('./models/adaboost_lstm.pkl') 

INFO:tensorflow:Assets written to: ram://4fc3a096-2c5e-42aa-8ed7-0d35dd718c6c/assets


INFO:tensorflow:Assets written to: ram://3f4d161d-ae84-45e1-b949-b9e25640463a/assets


INFO:tensorflow:Assets written to: ram://3f4d161d-ae84-45e1-b949-b9e25640463a/assets


INFO:tensorflow:Assets written to: ram://aea38500-abca-43c1-9f03-3d652beb35d0/assets


INFO:tensorflow:Assets written to: ram://aea38500-abca-43c1-9f03-3d652beb35d0/assets


...

INFO:tensorflow:Assets written to: ram://94288a7f-24f6-4f72-9a74-76d071b01a68/assets


INFO:tensorflow:Assets written to: ram://53037386-12b4-4385-b5cb-98f69aed3181/assets


INFO:tensorflow:Assets written to: ram://53037386-12b4-4385-b5cb-98f69aed3181/assets


INFO:tensorflow:Assets written to: ram://56c8a39b-d555-4124-a103-af0a24f06b89/assets


INFO:tensorflow:Assets written to: ram://56c8a39b-d555-4124-a103-af0a24f06b89/assets


## etc..

In [4]:
a = [i for i in boosted_classifier.staged_predict_proba(X_test)]
len(a[0])
a

[array([[0.14671701, 0.853283  ],
        [0.15134184, 0.84865814],
        [0.09548907, 0.9045109 ],
        ...,
        [0.12648392, 0.8735161 ],
        [0.10223823, 0.89776176],
        [0.23039924, 0.76960075]], dtype=float32),
 array([[0.8154902 , 0.18450987],
        [0.8216686 , 0.17833138],
        [0.0754172 , 0.9245828 ],
        ...,
        [0.20071441, 0.7992856 ],
        [0.06067883, 0.9393212 ],
        [0.0725329 , 0.9274671 ]], dtype=float32),
 array([[0.34155625, 0.6584438 ],
        [0.34451336, 0.6554866 ],
        [0.03459236, 0.9654076 ],
        ...,
        [0.08024294, 0.919757  ],
        [0.04676704, 0.95323294],
        [0.12316583, 0.8768342 ]], dtype=float32),
 array([[0.5092239 , 0.49077615],
        [0.60728794, 0.3927121 ],
        [0.16986866, 0.8301313 ],
        ...,
        [0.2854696 , 0.7145304 ],
        [0.20876725, 0.79123276],
        [0.24563938, 0.7543606 ]], dtype=float32),
 array([[0.53762466, 0.46237534],
        [0.3876752 , 0.6123248

In [8]:
[accuracy_score(y_test,i) for i in boosted_classifier.staged_predict(X_test)]

[0.7682847896440129,
 0.6220064724919094,
 0.7592233009708738,
 0.6517799352750809,
 0.7391585760517799,
 0.6660194174757281,
 0.7249190938511327,
 0.6724919093851133,
 0.7171521035598706,
 0.6964401294498382]

In [27]:
boosted_classifier.estimator_weights_

array([1., 1., 1., 1., 1.])

In [28]:
boosted_classifier.estimator_errors_

array([0.13636364, 0.18618371, 0.77088202, 0.80480714, 0.13529329])

In [100]:
y_pred_test

array([0, 0, 1, ..., 1, 1, 1])

In [101]:
boosted_classifier.estimators_

In [102]:
boosted_classifier.estimator_weights_

array([1., 1., 1., 1., 1.])

In [103]:
boosted_classifier.estimator_errors_

array([0.19136202, 0.19596451, 0.16152457, 0.14712343, 0.16450197])